In [42]:
import requests
from pprint import pprint
import json
import pandas as pd

API_KEY='5e5cec06054aaa4b312859dd97fb9a94c3211af602e0af17cd8350df3b15dbd1'

In [ ]:


# import requests

# Define the API endpoint to find locations in Kathmandu

url = "https://api.openaq.org/v3/locations?coordinates=27.7172,85.3240&radius=12000"

 
# headers
headers = {'X-API-Key': API_KEY}

# Define query parameters to get sensor id for Kathmandu
# params = {
#     "city": "Kathmandu",
#     "country": "NP",
#     "limit": 1
# }

# # Fetch data
response = requests.get(url,  headers=headers)
if response.status_code == 200:
    data = response.json()
    for location in data["results"]:
        pprint(location)
        
else:
    print(f"Failed to fetch data: {response}")


# Define the API endpoint to get data for a specific sensor
# List of sensor IDs for O₃
sensor_ids = [7713, 7712]  # Embassy Kathmandu and US Diplomatic Post: Phora Durbar Kathmandu

# # Fetch data for each sensor
# for sensor_id in sensor_ids:
#     url = f"https://api.openaq.org/v3/sensors/{sensor_id}/measurements/daily?date_from=2023-01-01&date_to=2023-01-31&limit=1000"

#     response = requests.get(url, headers=headers)
#     if response.status_code == 200:
#         data = response.json()
#         with open(f"sensor_{sensor_id}_daily_o3.json", "w") as file:
#             json.dump(data["results"], file, indent=4)
#         print(f"Data saved for sensor {sensor_id}")
#     else:
#         print(f"Failed to fetch data for sensor {sensor_id}: {response.status_code}")
        
# import requests



{'meta': {'name': 'openaq-api',
  'website': '/',
  'page': 1,
  'limit': 100,
  'found': 0},
 'results': []}

In [219]:


# Define the sensor ID for O₃ at US Diplomatic Post: Phora Durbar Kathmandu
sensor_id = 7712  # Replace with the correct sensor ID for O₃

# Define query parameters
params = {
    "datetime_from": "2020-12-28T00:00:00Z",  # Start date in ISO 8601 format
    "datetime_to": "2022-01-02T23:59:59Z",  # End date in ISO 8601 format
    "limit": 1000,  # Number of results per page
    "page": 1  # Start with page 1
}

# Define the API endpoint for daily measurements
url = f"https://api.openaq.org/v3/sensors/{sensor_id}/measurements/daily"


# Fetch all pages of data
all_results = []  # Store all results here
while True:
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        data = response.json()
        all_results.extend(data["results"])  # Add results to the list

        # Check if there are more pages
        if data["meta"]["found"] <= params["page"] * params["limit"]:
            break  # Exit the loop if all data has been fetched

        params["page"] += 1  # Move to the next page
    else:
        print(f"Failed to fetch data: {response.status_code}")
        print(f"Response: {response.text}")  # Print the error response for debugging
        break

print(f"Total daily measurements fetched: {len(all_results)}")

Total daily measurements fetched: 330


In [220]:

# extract the dates
dates = [result["period"]["datetimeFrom"]['utc'] for result in all_results]


len(dates)

dates = pd.to_datetime(dates).date # convert to datetime object and extract date

dates.min(), dates.max()



(datetime.date(2020, 12, 27), datetime.date(2022, 1, 2))

In [221]:


o3_values = [result["summary"]["median"] for result in all_results] # extract the median values

In [222]:
len(o3_values) # check the length of the values



330

In [223]:
# create a dataframe containing the dates and O₃ values
df = pd.DataFrame({"Date": dates[1:], "o3": o3_values[1:]})
df['o3'] = df['o3'].apply(lambda x: f"{x:.4f}")
df['Date'] = pd.to_datetime(df['Date']).dt.date
df.shape

(329, 2)

In [225]:
dataset = pd.read_csv("../year_2021/Kathmandu_2021.csv")
# dataset['o3'] = df['o3'].apply(lambda x: f"{x:.4f}")
dataset['Date'] = pd.to_datetime(dataset['Date']).dt.date

# dropping the last row
dataset = dataset.iloc[:,:-1]




dataset.shape

(371, 10)

In [226]:
# Merge the two datasets on the date column
merged_df= pd.merge(dataset,df, on="Date", how="left")

In [227]:
merged_df.sample(5)

,Date,dew,humidity,pm1,pm10,pm25,pressure,temperature,wind-gust,wind-speed,o3
176,2021-06-22,21.0,78.1,40.0,22.0,48.0,1011.0,24.3,NaN,1.1,NaN
137,2021-05-14,14.0,72.3,69.0,35.0,75.0,1011.0,20.0,NaN,0.8,0.0430
359,2021-12-22,4.0,76.2,152.0,64.0,157.0,1018.0,8.8,NaN,0.9,NaN
29,2021-01-26,3.5,72.5,NaN,NaN,172.0,1018.0,7.0,NaN,1.5,0.0050
312,2021-11-05,11.0,82.2,74.0,29.0,93.0,1018.0,14.6,NaN,0.8,0.0095


In [229]:
# convert the merged dataset to a csv file
merged_df.to_csv("../year_2021/Kathmandu_2021.csv", index=False)

In [240]:
import requests

# Define the coordinates for Pokhara, Nepal
latitude = 26.4525 
longitude = 87.2718
radius = 12000  # Radius in meters (e.g., 12 km)

# Define the API endpoint for locations
url = "https://api.openaq.org/v3/locations"

# Define query parameters
params = {
    "coordinates": f"{latitude},{longitude}",
    "radius": radius,
    "limit": 1000  # Number of results per page
}

# Make a GET request to the API
response = requests.get(url, params=params, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Extract the required fields
    locations = data["results"]
    for location in locations:
        print(f"Location: {location['name']}")
        print(f"ID: {location['id']}")
        print(f"Coordinates: {location['coordinates']}")
        print(f"Sensors: {location['sensors']}")
        print("-" * 40)
else:
    print(f"Failed to fetch data: {response.status_code}")
    print(f"Response: {response.text}")  # Print the error response for debugging
    
    

In [241]:
data

{'meta': {'name': 'openaq-api',
  'website': '/',
  'page': 1,
  'limit': 1000,
  'found': 0},
 'results': []}